In [2]:
import contextily as ctx
import folium
import geopandas as gpd
import pandas as pd
import requests
import leafmap.foliumap as leafmap
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from mapclassify import classify
from folium.plugins import GroupedLayerControl


# Préparation des données géographiques des quartiers

In [3]:
# Importation des données géographiques des quartiers de Marseille
url = 'https://www.data.gouv.fr/api/1/datasets/r/8a8f7f54-7f91-482c-a78c-dd09d893d1b6'
file = requests.get(url)
data = file.content
quartiers_data = gpd.read_file(data)

In [4]:
quartiers_data.info()
quartiers_data.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   DEPCO     111 non-null    object  
 1   NOM_CO    111 non-null    object  
 2   NOM_QUA   111 non-null    object  
 3   geometry  111 non-null    geometry
dtypes: geometry(1), object(3)
memory usage: 3.6+ KB


,DEPCO,NOM_CO,NOM_QUA,geometry
0,13201,Marseille 1er Arrondissemen,BELSUNCE,"MULTIPOLYGON (((5.38086 43.29924, 5.38087 43.2..."
1,13201,Marseille 1er Arrondissemen,CHAPITRE,"MULTIPOLYGON (((5.38525 43.29906, 5.38485 43.2..."
2,13201,Marseille 1er Arrondissemen,NOAILLES,"MULTIPOLYGON (((5.3816 43.29573, 5.38177 43.29..."
3,13201,Marseille 1er Arrondissemen,OPERA,"MULTIPOLYGON (((5.37729 43.29222, 5.37663 43.2..."
4,13201,Marseille 1er Arrondissemen,SAINT CHARLES,"MULTIPOLYGON (((5.38022 43.30141, 5.38007 43.3..."


In [5]:
quartiers_data.loc[quartiers_data['NOM_QUA'] == 'ENDOUME']

,DEPCO,NOM_CO,NOM_QUA,geometry
29,13207,Marseille 7e Arrondissemen,ENDOUME,"MULTIPOLYGON (((5.34923 43.28452, 5.34919 43.2..."
30,13207,Marseille 7e Arrondissemen,ENDOUME,"MULTIPOLYGON (((5.34756 43.29063, 5.3489 43.28..."


In [6]:
# l'observation 30 correspond aux îles
quartiers_data.iloc[30]['NOM_QUA'] = 'LES ILES'

# Préparation des données sur les pav

In [7]:
pav_marseille = gpd.read_file('../data/raw/pav_marseille_fixed.geojson')
pav_marseille.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5184 entries, 0 to 5183
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   OBJECTID             5184 non-null   int32   
 1   Code Commune INSEE   5184 non-null   float64 
 2   Quartier             5184 non-null   object  
 3   Volume intérieur m3  3378 non-null   float64 
 4   Type de flux         5184 non-null   object  
 5   longitude            5184 non-null   float64 
 6   latitude             5184 non-null   float64 
 7   geometry             5184 non-null   geometry
dtypes: float64(4), geometry(1), int32(1), object(2)
memory usage: 303.9+ KB


# Préparation des données de population

In [8]:
#import des données de population des quartiers de Marseille
df_pop_quartier = pd.read_csv("../data/raw/pop_marseille_2022.csv", sep=";")

# Traitement des dataframes

## Jointure entre les données des quartiers et les données de population

In [9]:
df_pop_quartier['quartier'] = df_pop_quartier['quartier'].apply(lambda x : x.replace('-', ' ').upper())

In [10]:
# On va modifier certains noms de quartiers pour qu'ils correspondent entre les deux DataFrames
mapping = {
    'BAUMETTES': 'LES BAUMETTES',
    'BLANCARDE': 'LA BLANCARDE',
    'BORELS': 'LES BORELS',
    'CAMAS': 'LE CAMAS',
    'CAPELETTE': 'LA CAPELETTE',
    'CHARTREUX': 'LES CHARTREUX',
    'CHATEAU-GOMBERT' : 'CHATEAU GOMBERT',
    'CHÂTEAU GOMBERT' : 'CHATEAU GOMBERT',
    'CHUTES LAVIE' : 'LES CHUTES LAVIE',
    'LES CHUTES LAVIES' : 'LES CHUTES LAVIE',
    'CINQ AVENUES' : 'LES CINQ AVENUES',
    'CONCEPTION': 'LA CONCEPTION',
    'CROIX ROUGE': 'LA CROIX ROUGE',
    'LA FOURRAGÈRE' : 'LA FOURRAGERE',
    'GRANDS CARMES' : 'LES GRANDS CARMES',
    'HÔTEL DE VILLE' : 'HOTEL DE VILLE',
    'LA MILLIÈRE' : 'LA MILLIERE',
    'MALPASSÉ' : 'MALPASSE',
    'LES MÉDECINS' : 'LES MEDECINS',
    'MOURETS': 'LES MOURETS',
    'OPÉRA' : 'OPERA',
    'PÉRIER' : 'PERIER',
    'PHARO': 'LE PHARO',
    'POINTE ROUGE' : 'LA POINTE ROUGE',
    'PRÉFECTURE' : 'PREFECTURE',
    'REDON': 'LE REDON',
    'ROUCAS BLANC' : 'LE ROUCAS BLANC',
    'LE ROUCAS' : 'LE ROUCAS BLANC',
    'ROUET': 'LE ROUET',
    'SAINT ANDRÉ' : 'SAINT ANDRE',
    'SAINT BARNABÉ' : 'SAINT BARNABE',
    'ST BARNANE' : 'SAINT BARNABE',
    'SAINT BARTHÉLEMY' : 'SAINT BARTHELEMY',
    'SAINT JEAN DU DÉSERT' : 'SAINT JEAN DU DESERT',
    'SAINT JÉRÔME' : 'SAINT JEROME',
    'SAINT MAURON': 'SAINT MAURONT',
    'STE MARGUERITE' : 'SAINTE MARGUERITE',
    'VAUFRÈGES' : 'VAUFREGES',
    'VIELLE CHAPELLE': 'VIEILLE CHAPELLE',
    'VILETTE': 'LA VILLETTE',
    'VILLETTE' : 'LA VILLETTE',
    'LA VILETTE' : 'LA VILLETTE',
}

In [11]:
df_pop_quartier['quartier'] = df_pop_quartier['quartier'].replace(mapping)
quartiers_data['NOM_QUA'] = quartiers_data['NOM_QUA'].replace(mapping)
pav_marseille['Quartier'] = pav_marseille['Quartier'].replace(mapping)

In [12]:
quartiers_merge = df_pop_quartier.merge(quartiers_data, left_on='quartier', right_on = 'NOM_QUA').drop('NOM_QUA', axis = 1)

In [13]:
quartiers_merge = quartiers_merge.rename({'quartier' : 'Quartier'}, axis = 1)
quartiers_merge

,Quartier,population,DEPCO,NOM_CO,geometry
0,BELSUNCE,8825,13201,Marseille 1er Arrondissemen,"MULTIPOLYGON (((5.38086 43.29924, 5.38087 43.2..."
1,CHAPITRE,6497,13201,Marseille 1er Arrondissemen,"MULTIPOLYGON (((5.38525 43.29906, 5.38485 43.2..."
2,NOAILLES,4304,13201,Marseille 1er Arrondissemen,"MULTIPOLYGON (((5.3816 43.29573, 5.38177 43.29..."
3,OPERA,5468,13201,Marseille 1er Arrondissemen,"MULTIPOLYGON (((5.37729 43.29222, 5.37663 43.2..."
4,SAINT CHARLES,8390,13201,Marseille 1er Arrondissemen,"MULTIPOLYGON (((5.38022 43.30141, 5.38007 43.3..."
...,...,...,...,...,...
106,LA VISTE,6665,13215,Marseille 15e Arrondissemen,"MULTIPOLYGON (((5.35771 43.35337, 5.35756 43.3..."
107,L'ESTAQUE,5807,13216,Marseille 16e Arrondissemen,"MULTIPOLYGON (((5.32691 43.36112, 5.32694 43.3..."
108,LES RIAUX,722,13216,Marseille 16e Arrondissemen,"MULTIPOLYGON (((5.30862 43.36508, 5.30878 43.3..."
109,SAINT ANDRE,3739,13216,Marseille 16e Arrondissemen,"MULTIPOLYGON (((5.33997 43.34367, 5.33992 43.3..."


## Jointure des autres dataframe

In [14]:
pav_quartiers = pav_marseille.copy()

In [15]:
# conversion en geodataframe
pav_quartiers = gpd.GeoDataFrame(
    pav_quartiers,
    geometry=gpd.points_from_xy(
        pav_quartiers['longitude'],
        pav_quartiers['latitude'],
        crs="EPSG:4326"))

pav_quartiers = pav_quartiers.to_crs(3857)

In [16]:
# Trop de données manquantes sur le volume de points d'apport à l'échelle du quartier...
print(pav_quartiers.isna().mean())


OBJECTID               0.00000
Code Commune INSEE     0.00000
Quartier               0.00000
Volume intérieur m3    0.34838
Type de flux           0.00000
longitude              0.00000
latitude               0.00000
geometry               0.00000
dtype: float64


In [17]:
# On se contente de récupérer le nombre de points d'apport par quartier
nb_pav_quartiers = pav_quartiers.groupby(['Quartier', 'Type de flux']).count()['OBJECTID'].reset_index()
nb_pav_quartiers

,Quartier,Type de flux,OBJECTID
0,ARENC,Biflux,7
1,ARENC,Biodechets,1
2,ARENC,OM,1
3,ARENC,Textile,1
4,ARENC,Verre,7
...,...,...,...
455,VERDURON,Verre,4
456,VIEILLE CHAPELLE,Biflux,13
457,VIEILLE CHAPELLE,Biodechets,8
458,VIEILLE CHAPELLE,Textile,1


In [18]:
types_flux = pav_quartiers['Type de flux'].unique()
types_flux

array(['Verre', 'Biflux', 'OM', 'Carton', 'Biodechets', 'Textile'],
      dtype=object)

In [19]:
# ajout des lignes pour lesquelles la combinaison quartier/types de flux n'existe pas
# afin que ces quartiers apparaissent tout de même sur la carte, avec une densité de 0
for quartier in nb_pav_quartiers['Quartier'].unique():
    quartier_df = nb_pav_quartiers.loc[nb_pav_quartiers['Quartier'] == quartier]
    for type in types_flux:
        if type not in quartier_df['Type de flux'].unique():
            new_line = pd.DataFrame.from_dict(
                {
                    'Quartier' : [quartier],
                    'Type de flux' : [type],
                    'OBJECTID' : [0]
                }
            )
            nb_pav_quartiers = pd.concat([nb_pav_quartiers, new_line])

In [20]:
# merge des données de carte, de population et de volume des points d'apport
marseille = (quartiers_merge
             .merge(nb_pav_quartiers, on = 'Quartier', how = 'right')
             .rename({'OBJECTID' : 'nb_points'}, axis = 1)
)

marseille['nb_points'] = marseille['nb_points'].fillna(0).astype(int)



# On passe en Geodataframe pour les cartes

In [21]:
#Conversion de marseille en geodataframe
gdf_marseille = gpd.GeoDataFrame(marseille, geometry='geometry').to_crs(3857)

In [22]:
# Calcul de la densité des points d'apport par quartier
gdf_marseille['pav_pour_1000_hab'] = gdf_marseille['nb_points']/gdf_marseille['population']*1000

In [23]:
# ajout de classes d'intervalles pour uniformiser les couleurs
# choix des classes d'intervalles
taux_max = max(gdf_marseille['pav_pour_1000_hab'])
ref_bins = [0, 0.5, 1, 2, 3, 4, 5, taux_max]
ref_labels = ['< 0,5', '(0.5, 1]', '(1, 2]', '(2, 3]', '(3, 4]', '(4, 5]', '> 5']

# création d'un dictionnaire de couleurs pour chaque classe d'intervalle à partir de la cmap 'Oranges'
n_bins = len(ref_bins)-1
cmap = mpl.colormaps['Greens']
ref_colors = cmap(np.linspace(0, 1, n_bins))
custom_cmap = mpl.colors.ListedColormap(ref_colors)


color_dict = {}
color_dict[np.nan] = '#808080'
for n in range(n_bins):
    color_dict[ref_labels[n]] = mpl.colors.rgb2hex(custom_cmap(n))

In [24]:
gdf_marseille['label'] = pd.cut(gdf_marseille['pav_pour_1000_hab'], bins = ref_bins, labels = ref_labels)
gdf_marseille['color'] = gdf_marseille['label'].map(color_dict)
gdf_marseille

,Quartier,population,DEPCO,NOM_CO,geometry,Type de flux,nb_points,pav_pour_1000_hab,label,color
0,ARENC,1668,13202,Marseille 2e Arrondissemen,"MULTIPOLYGON (((597689.222 5360305.255, 597754...",Biflux,7,4.196643,"(4, 5]",#0b7734
1,ARENC,1668,13202,Marseille 2e Arrondissemen,"MULTIPOLYGON (((597689.222 5360305.255, 597754...",Biodechets,1,0.599520,"(0.5, 1]",#dbf1d6
2,ARENC,1668,13202,Marseille 2e Arrondissemen,"MULTIPOLYGON (((597689.222 5360305.255, 597754...",OM,1,0.599520,"(0.5, 1]",#dbf1d6
3,ARENC,1668,13202,Marseille 2e Arrondissemen,"MULTIPOLYGON (((597689.222 5360305.255, 597754...",Textile,1,0.599520,"(0.5, 1]",#dbf1d6
4,ARENC,1668,13202,Marseille 2e Arrondissemen,"MULTIPOLYGON (((597689.222 5360305.255, 597754...",Verre,7,4.196643,"(4, 5]",#0b7734
...,...,...,...,...,...,...,...,...,...,...
649,VAUFREGES,610,13209,Marseille 9e Arrondissemen,"MULTIPOLYGON (((607860.861 5343975.331, 607854...",Carton,0,0.000000,NaN,#808080
650,VERDURON,8301,13215,Marseille 15e Arrondissemen,"MULTIPOLYGON (((594771.038 5367762.587, 594720...",Carton,0,0.000000,NaN,#808080
651,VERDURON,8301,13215,Marseille 15e Arrondissemen,"MULTIPOLYGON (((594771.038 5367762.587, 594720...",Textile,0,0.000000,NaN,#808080
652,VIEILLE CHAPELLE,7945,13208,Marseille 8e Arrondissemen,"MULTIPOLYGON (((598878.99 5349077.238, 598877....",OM,0,0.000000,NaN,#808080


## Carte du nombre de point d'apport

In [25]:
legend_colors = color_dict
if np.nan in legend_colors.keys():
    legend_colors.pop(np.nan)

In [26]:
# fond de carte
map = leafmap.Map(
    location = [43.275, 5.4],
    tiles = 'cartodbpositron',
    zoom_start = 12,
    max_zoom=20,
    control_scale=True
    )



# ajout des couches pour chaque type de flux
layers = {}

for type in types_flux:
    layer_name = 'fg_'+ type 
    label = type
    layers[layer_name] = folium.FeatureGroup(name = label).add_to(map)

    
    pav_quartiers.loc[pav_quartiers['Type de flux'] == type].explore(
        m = layers[layer_name],
        marker_kwds=dict(radius=2, fill=True),
        color = 'Red'
    )


    gdf_marseille.loc[gdf_marseille['Type de flux'] == type].explore(
    m = layers[layer_name], 
    color = 'color',
    tooltip = ['Quartier', 'population', 'nb_points', 'pav_pour_1000_hab'],
    popup = True,
    cmap = 'Greens',
    legend = False,
    style_kwds = dict(
            fillOpacity = 0.9,
            color = 'green',
            opacity = 0.5),
        highlight_kwds = dict(
            fillOpacity = 1)
            )



    map.add_child(layers[layer_name])





# Contrôle de l'affichage des couches
GroupedLayerControl(
    groups={'Type de flux': [layer for layer in layers.values()]},
    collapsed=False,
).add_to(map)




# ajout de la légende
legend_style = {
    "position": "fixed",
    "z-index": "9999",
    "border": "2px solid grey",
    "background-color": "rgba(255, 255, 255, 0.8)",
    "border-radius": "10px",
    "padding": "5px",
    "font-size": "14px",
    "bottom": "20px",
    "right": "5px",
}

map.add_legend(
    title="nb pav/1000 hab.", legend_dict=legend_colors, draggable=False, style=legend_style
)


# ajout du titre
map_title = "Nombre de pav pour 1000 hab"
title_html = f'<h1 align="center" style="font-size:24px" >{map_title}</h1>'
map.get_root().html.add_child(folium.Element(title_html))


map.save('../cartes/map_densite_pav_pop.html')

